## O problema do Mix de Produção

Vamos utilizar o exemplo da página 26 do livro Pesquisa Operacional para Cursos de Engenharia (Belfiore e Fávero, 2013).

A empresa Venix de brinquedos está revendo seu planejamento de produção de carrinhos e triciclos. O lucro líquido por unidade de carrinho e triciclo produzido é de $\text{R\$ } 12,00$ e $\text{R\$ } 60,00$, respectivamente. 

As matérias-primas e os insumos necessários para a fabricação de cada um dos produtos são terceirizados, cabendo à empresa os processos de usinagem, pintura e montagem. O processo de usinagem requer 15 minutos de mão de obra especializada por unidade de carrinho e 30 minutos por unidade de triciclo produzida. 

O processo de pintura requer 6 minutos de mão de obra especializada por unidade de carrinho e 45 minutos por unidade de triciclo produzida. 

Já o processo de montagem necessita de 6 minutos e 24 minutos para uma unidade de carrinho e de triciclo produzida, respectivamente. 

O tempo disponível por semana é de 36, 22 e 15 horas para os processos de usinagem, pintura e montagem, respectivamente. 

A empresa quer determinar quanto produzir de cada produto por semana, respeitando as limitações de recursos, de forma a **maximizar o lucro líquido semanal**.

## Formulação do modelo matemático

### Variáveis de decisão

$x_1 = \text{quantidade de carrinhos a ser fabricada por semana}$

$x_2 = \text{quantidade de tricilos a ser fabricada por semana}$

### Função objetivo

$\max z = 12x_1 + 60x_2$

### Restrições


#### Restrições de disponibilidade de mão de obra

$0,25x_1 + 0,5x_2 \leq 36$

$0,1x_1 + 0,75x_2 \leq 22$

$0,1x_1 + 0,4x_2 \leq 15$

#### Restrições de não negatividade

$x_1 \geq 0$

$x_2 \geq 0$

## Representação matricial

### Função objetivo

$\max z = \mathbf{c}^T\mathbf{x}$

### Restrições

$\mathbf{Ax} \leq \mathbf{b}$

$\mathbf{x} \geq 0$

## Representação genérica

### Índices:

$i = 1, \ldots, m$

$j = 1, \ldots, n$

### Parâmetros:

$c_j = \text{lucro líquido por unidade do produto } j \text{ produzido}$

$a_{ij} = \text{quantidade de horas necessárias para a execução do processo } i \text{ na fabricação do produto } j$

$b_i = \text{quantidade de horas disponíveis para execução do processo } i$

### Variáveis de decisão:

$x_j = \text{quantidade a ser fabricada do produto } j \text{ por semana}$

### Modelo matemático:

$\text{max }z = \sum\limits_{j=1}^{n} c_j x_{j}$

sujeito a:

$\sum\limits_{j=1}^{n} a_{ij} x_{j} \leq b_{i}, \forall \; i$

$x_{j} \geq 0$

## Modelo computacional

In [1]:
c = [12,60]

a = [[0.25, 0.5], 
     [0.1, 0.75], 
     [0.1, 0.4]]

b = [36, 22, 15]

In [2]:
import pyomo.environ as pyo

In [3]:
modelo = pyo.ConcreteModel()

In [4]:
modelo.m = range(len(a))
modelo.n = range(len(a[0][:]))

In [5]:
modelo.c = pyo.Param(modelo.n, initialize=lambda modelo, j: c[j])
modelo.a = pyo.Param(modelo.m, modelo.n, initialize=lambda modelo, i, j: a[i][j])
modelo.b = pyo.Param(modelo.m, initialize=lambda modelo, i: b[i])

In [6]:
modelo.x = pyo.Var(modelo.n, within=pyo.NonNegativeReals)

In [7]:
def f_obj(modelo):
    return pyo.summation(modelo.c, modelo.x)

modelo.z = pyo.Objective(rule=f_obj, sense=pyo.maximize)

In [8]:
def f_constr(modelo, i):
    return sum(modelo.a[i,j] * modelo.x[j] for j in modelo.n) <= modelo.b[i]

modelo.restr_horas = pyo.Constraint(modelo.m, rule=f_constr)

In [9]:
resultado = pyo.SolverFactory('gurobi', solver_io="python").solve(modelo)

Academic license - for non-commercial use only - expires 2021-06-07
Using license file /Users/acsjunior/gurobi.lic


In [10]:
l = list(modelo.x.keys())
for i in l:
    print("x" + str(i+1),'=', modelo.x[i]())

x1 = 70.0
x2 = 20.0


In [11]:
modelo.z()

2040.0